In [15]:
# Import libraries
import os
import numpy as np
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from pybevc import PyBEVC
from utils import save_load
from inertial_forces import inertial_loads_fun_v03

In [16]:
# Main loop options
epsilon = 1e-3
delta_u_rel = epsilon + 1
iter_max = 20

In [17]:
# Simulation conditions
U0 = 10
TSR = 7 #Not used for now
omega = 1 # rad/s
pitch_deg = 0
pitch_rad = np.deg2rad(pitch_deg)
hub_di = 3 # See how to get it from htc file

In [18]:
# Folders
htcfolder = 'iea15mw'
projectfolder = 'project_folder'

## Inertial model

In [ ]:
# Input files folder
propfile = os.path.join(os.getcwd(),'property_file.dat')

stPropertyInput = np.loadtxt(propfile, skiprows=3)

# cg_offset are a set of vectors in blade root coordinates that indicate the position of the CoG with
# respect to then nodes (centre of elasticity)
# r_node2cg = r_c22cg - r_c22node
cg_offset = 

## Structural model init

In [19]:
# Model input json file  name
f_model_json = "iea15mw_toy_model.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'iea15mw')
mainfile = os.path.join(inputfolder,f_model_json)

# Initialize beam model
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam = ComplBeam(mainfile)


-----------
ComplBeam Model Created
Static analysis done


In [20]:
# For non dynamic calculations, the mass matrix is not calculated
f_model_json = "iea15mw_dynamic.json"
inputfolder = os.path.join(os.getcwd(),'iea15mw')
mainfile = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix and apply to static beam
beam_dynamic = ComplBeam(mainfile)
beam.M_mat_full = beam_dynamic.M_mat_full

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


## Aerodynamic model init

In [21]:
bevc = PyBEVC()

# Setting inputs manually 
bevc.U0 = 8.0
bevc.TSR = 7


bevc.flag_a_CT = 2

htc_filename = 'iea15mw_bevc/IEA-15-240-RWT-Onshore/htc/IEA_15MW_RWT_Onshore.htc'
bevc.from_htc_file(htc_filename, model_path='data') # HAWC2 HTC file

# Display all data as an Xarray object
xr_inp = bevc.as_xarray()
xr_inp

<xarray.Dataset> Size: 317kB
Dimensions:         (s_polar: 39, aoa: 200, s: 34)
Coordinates:
  * s_polar         (s_polar) float64 312B 21.1 21.27 21.5 ... 96.27 99.04 100.0
  * aoa             (aoa) float64 2kB -180.0 -177.0 -174.0 ... 174.0 177.0 180.0
  * s               (s) float64 272B 0.0 0.01001 0.02002 ... 0.9003 0.9502 1.0
Data variables: (12/36)
    NB              int64 8B 3
    R               float64 8B 120.7
    TSR             int64 8B 7
    U0              int64 8B 12
    aoa_deg         (s_polar, aoa) float64 62kB -180.0 -177.0 ... 177.0 180.0
    aoa_rad         (s_polar, aoa) float64 62kB -3.142 -3.089 ... 3.089 3.142
    ...              ...
    twist_deg       (s) float64 272B -15.59 -15.59 -15.59 ... 2.093 1.8 1.242
    twist_rad       (s) float64 272B -0.2722 -0.2721 -0.2721 ... 0.03142 0.02168
    x               (s) float64 272B 0.02277 -0.01294 ... -0.2989 -0.06589
    x_cp            float64 8B 0.5
    y               (s) float64 272B -0.006354 0.01262 0.03159 ... -3.444 -4.001
    z               (s) float64 272B 0.0 1.171 2.342 3.513 ... 105.3 111.2 117.0

In [22]:
# - Init variables
pos_new = np.zeros(beam.M_mat_full.shape[0]) # The initial pos are 0. This gives no inertial forces
tip_init_pos=beam.nodeLocations[-1]
old_tip_def = np.zeros(3)
new_tip_def = np.zeros(3)
tip_def_history = []
delta_history = []
iter = 0

while abs(delta_u_rel) > epsilon and iter < iter_max:
    # Update variables
    pos_old = pos_new
    old_tip_def = new_tip_def
    tip_def_history.append(np.linalg.norm(old_tip_def))
    iter += 1

    # Calculate inertial loads
    inertial_loads = inertial_loads_fun_v03(pos_new, beam, beam.M_mat_full, hub_di, omega, pitch_rad)
    save_load(inertial_loads,projectfolder)

    # Calculate delta
    delta_u_rel = np.linalg.norm(new_tip_def - old_tip_def)/r[-1]
    delta_history.append(delta_u_rel)

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'